In [1]:
import time

In [8]:
a = time.time()

In [10]:
b = time.time()

In [12]:
b-a

13.31199598312378

In [16]:
request_time_stamps = []
for i in range(100):
    time.sleep(0.5)
    request_time_stamps.append(time.time())
    #print(f"Length of list: {len(request_time_stamps)}.")
    if len(request_time_stamps) > 30:
        first = request_time_stamps.pop(0)
        print(time.time() - first)
        if time.time() - first < 30:
            print(f"SLEEP for {request_time_stamps[0]- first}")
            time.sleep(request_time_stamps[0]- first)

15.01578974723816
SLEEP for 0.5005981922149658
15.51721739768982
SLEEP for 0.5005848407745361
16.01875901222229
SLEEP for 0.5005893707275391
16.519999742507935
SLEEP for 0.5006136894226074
17.021355390548706
SLEEP for 0.5005943775177002
17.52226448059082
SLEEP for 0.5006005764007568
18.023983001708984
SLEEP for 0.5001916885375977
18.525282382965088
SLEEP for 0.50010085105896
19.02668046951294
SLEEP for 0.500584602355957
19.528099298477173
SLEEP for 0.5005404949188232
20.02983784675598
SLEEP for 0.5005440711975098
20.5311017036438
SLEEP for 0.500542163848877
21.032055854797363
SLEEP for 0.5005767345428467
21.53338623046875
SLEEP for 0.5005385875701904
22.034688472747803
SLEEP for 0.5003273487091064
22.53637170791626
SLEEP for 0.500394344329834
23.037362813949585
SLEEP for 0.5002861022949219
23.538630723953247
SLEEP for 0.5005447864532471
24.03917932510376
SLEEP for 0.500582218170166
24.54013419151306
SLEEP for 0.500537633895874
25.041473627090454
SLEEP for 0.500605583190918
25.542816638